In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import seaborn as sns
from sklearn.feature_extraction import FeatureHasher
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import scipy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style="darkgrid")
pd.set_option('display.max_columns', 0)
plt.style.use('ggplot')
pd.options.display.float_format = '{:.2f}'.format
import math
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
import random
import gzip
import category_encoders as ce
pd.options.display.float_format = '{:.2f}'.format


import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

!pip install -U deepctr-torch

from deepctr_torch.inputs import SparseFeat, get_feature_names,DenseFeat
from deepctr_torch.models import DeepFM
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is 

1. train - Training set. 10 days of click-through data, ordered chronologically. Non-clicks and clicks are subsampled according to different strategies.
2. test - Test set. 1 day of ads to for testing your model predictions. 
3. sampleSubmission.csv - Sample submission file in the correct format, corresponds to the All-0.5 Benchmark.

# Data fields
1. id: ad identifier- **4577464** unqiue values
2. click: 0/1 for non-click/click-  **[0,1]** binary values
3. hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC - **29** unqiue hours
4. C1 -- anonymized categorical variable - **7** categorical values,
5. banner_pos- **7** unique positions, these are **ordinal**. It is acceptable as position is ordinal
6. site_id- **2962** unique id's
7. site_domain- **3542** unqiue values
8. site_category- **22** unique values
9. app_id- **4333** unique values
10. app_domain- **304** unique values
11. app_category- **31** unqiue values
12. device_id- **408547** unique values
13. device_ip- **1189783** unique values
14. device_model- **6195** unique values
15. device_type- **4** unique values, these are **ordinal**, It is acceptable as device compute/size can be ordinal
16. device_conn_type- **4** unique values, these are **ordinal**, It is acceptable as device connection can be slow and fast i.e ordinal.
17. C14-C21 -- anonymized categorical variables
- these all are **ordinal**
- C14 - (792,)
- C15 - (8,)
- C16 - (9,)
- C17 - (194,)
- C18 - (4,)
- C19 - (45,)
- C20 - (168,)
- C21 - (38,)


In [ ]:
# trainDf = pd.read_csv('/kaggle/input/avazu-ctr-prediction/train.gz',compression='gzip', nrows =4577464, header=0, sep=',', quotechar='"')
# testDf = pd.read_csv('/kaggle/input/avazu-ctr-prediction/test.gz',compression='gzip', nrows =4577464, header=0, sep=',', quotechar='"')


In [63]:
# trainDf.shape,testDf.shape
!ls ../input/avazu-ctr-prediction/

sampleSubmission.gz  test.gz  train.gz


In [10]:
num_records = 40428967
sample_size = 5000000
skip_values = sorted(random.sample(range(1,num_records), num_records - sample_size))
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')



train = pd.read_csv( "../input/avazu-ctr-prediction/train.gz", parse_dates=['hour'],chunksize=500000, date_parser=parse_date,skiprows=skip_values)
train=next(train)

test=pd.read_csv( "../input/avazu-ctr-prediction/test.gz", parse_dates=['hour'], date_parser=parse_date)

test= next(test)

TypeError: 'DataFrame' object is not an iterator

In [11]:
train.shape,test.shape

((500000, 24), (4577464, 23))

In [44]:
def preprocess(df,std_df):
    
    datecolumn=['hour']
    for col in datecolumn:
        df[col+'month'] = df[col].dt.month
        df[col+'dayofweek'] = df[col].dt.dayofweek
        df[col+'day'] = df[col].dt.day
        df[col+'hour_time'] = df[col].dt.hour
    col = ['C15', 'C16', 'C19', 'C21']
    for col in col:
        percentiles = df[col].quantile(0.98)
        if df[col].quantile(0.98) < 0.5 * df[col].max():
            df[col][df[col] >= percentiles] = percentiles
    datecolumn=['hour']
    for col in datecolumn:
        df[col+'month'] = df[col].dt.month
        df[col+'dayofweek'] = df[col].dt.dayofweek
        df[col+'day'] = df[col].dt.day
        df[col+'hour_time'] = df[col].dt.hour
        
    label='click'
    target = [label]
    dense_features = []
    sparse_features = []
    for col in (list(df.columns)):
        if df[col].dtype == "object":
            sparse_features.append(col)
        else:
            dense_features.append(col)
#     print(dense_features)
#     dense_features.remove(label)
    df[sparse_features] = df[sparse_features].fillna('-1', )
    df[dense_features] = df[dense_features].fillna(0, )
    drop_unique_ids=[]
#     for col in (list(df.columns)):
#         if len(list(df[col].unique())) == df.shape[0]:
#             drop_unique_ids.append(col)
#         if(len(drop_unique_ids)>0):
#             print(drop_unique_ids)
#             df.drop(drop_unique_ids, axis=1, inplace=True)
    print(dense_features)
#     dense_features.remove('id')
    dense_features.remove('hour')
    dense_features.remove('C17')
    for feat in sparse_features:
        print(feat)
        no_of_feat=std_df[feat].unique().shape[0]
        hsh = FeatureHasher(n_features=no_of_feat,input_type="string")
        hsh=hsh.fit(np.array(std_df[feat]).reshape((-1,1)))
        hsh=hsh.transform(np.array(df[feat]).reshape((-1,1)))
        sparse_matrix = csr_matrix(hsh)
        scipy.sparse.save_npz(feat+"_sp",hsh)
        del hsh
    stdsc = StandardScaler().fit(std_df[dense_features])
    scipy.sparse.save_npz("dense_feat",csr_matrix(stdsc.transform(df[dense_features])))






In [33]:
train.head(2)

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,hourmonth,hourdayofweek,hourday,hourhour_time
0,1,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,37e8da74,5db079b5,1,2,15707,320,50,1722,0,35,-1,79,10,1,21,0
1,1,2014-10-21,1005,0,85f751fd,c4e18dd6,50e219e0,54c5d545,2347f47a,0f2161f8,9af87478,2a2bfc89,ecf10acf,1,0,21611,320,50,2480,3,297,100111,61,10,1,21,0


In [42]:
test.head(2)

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,hourmonth,hourdayofweek,hourday,hourhour_time
0,2014-10-31,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,69f45779,0eb711ec,1,0,8330,320,50,761,3,175,100075,23,10,4,31,0
1,2014-10-31,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8d44657,ecb851b2,1,0,22676,320,50,2616,0,35,100083,51,10,4,31,0


In [37]:
# !rm -rf /kaggle/working/*

In [45]:
preprocess(test,train)

['hour', 'C1', 'banner_pos', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hourmonth', 'hourdayofweek', 'hourday', 'hourhour_time']
site_id
site_domain
site_category
app_id
app_domain
app_category
device_id
device_ip
device_model


In [47]:
from scipy.sparse import csr_matrix,hstack
from scipy.sparse import load_npz

mt_1 = load_npz('/kaggle/working/site_id_sp.npz')
mt_2 = load_npz('/kaggle/working/site_domain_sp.npz')
mt_3 = load_npz('/kaggle/working/site_category_sp.npz')

mt_4 = load_npz('/kaggle/working/app_id_sp.npz')
mt_5 = load_npz('/kaggle/working/app_domain_sp.npz')

mt_6 = load_npz('/kaggle/working/app_category_sp.npz')

mt_7 = load_npz('/kaggle/working/device_id_sp.npz')

mt_8 = load_npz('/kaggle/working/device_ip_sp.npz')

mt_9 = load_npz('/kaggle/working/device_model_sp.npz')


dt=load_npz("/kaggle/working/dense_feat.npz")


In [48]:
scipy.sparse.save_npz("X_test",hstack([mt_1, mt_2,mt_3,mt_4,mt_5,mt_6,mt_7,mt_8,mt_9,dt]))